In [1]:
import requests   #引入requests
from lxml import etree   #引入lxml的etree
import datetime,time
import re
from tqdm import tqdm
import pandas as pd
def getBetweenDay(begin_date,end_date):
    date_list = []
    begin_date = datetime.datetime.strptime(begin_date,"%Y%m%d")
    end_date = datetime.datetime.strptime(end_date,"%Y%m%d")
    while begin_date <= end_date:
        date_str = begin_date.strftime("%Y%m%d")
        date_list.append(date_str)
        begin_date +=datetime.timedelta(days=1)
    return date_list

date_list=getBetweenDay("20190801","20191101")
pattern="[\u4e00-\u9fa5]+"
regex = re.compile(pattern)
results = []

#绍兴
for day in tqdm(date_list):
    result = []
    result.append(day)
    r = requests.get('http://www.tianqihoubao.com/lishi/shangyu/' + day + '.html')   #用get方法获取网页
    html = r.text   #把内容赋值给html
    s = etree.HTML(html)   #把html转换成lxml可解析的对象
    
    #白天风力风向
    day_wind = str(s.xpath('//*[@id="content"]/table/tr[4]/td[2]//text()')[0])   
    result.append(day_wind)
    #夜晚风力风向
    night_wind = str(s.xpath('//*[@id="content"]/table/tr[4]/td[3]/text()')[0])  
    result.append(night_wind)
    #白天气温
    day_tem = str(s.xpath('//*[@id="content"]/table/tr[3]/td[2]/b/text()')[0])   
    result.append(day_tem)
    #夜晚气温
    night_tem = str(s.xpath('//*[@id="content"]/table/tr[3]/td[3]/b/text()')[0])
    result.append(night_tem)
    #白天天气
    day_weather = regex.findall(str(s.xpath('//*[@id="content"]/table/tr[2]/td[2]/text()')[1]))[0]
    result.append(day_weather)
    #白天天气
    day_weather = regex.findall(str(s.xpath('//*[@id="content"]/table/tr[2]/td[3]/text()')[1]))[0]
    result.append(day_weather)
    results.append(result)

results = pd.DataFrame(results,columns=['日期','白天风向风力','夜晚风向风力','白天温度','夜晚温度','白天天气','夜晚天气'])
results.to_csv('C:/Users/Flash/Desktop/shangyuresult.csv',header=1,encoding='utf_8_sig',index=0) #保存列名

100%|██████████████████████████████████████████████████████████████████████████████████| 93/93 [05:12<00:00,  3.36s/it]


In [2]:
from selenium import webdriver
import time
import pandas as pd
from tqdm import tqdm
from datetime import datetime
# 要想调用键盘按键操作需要引入keys包
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
driver = webdriver.Chrome()
driver.get("http://www.tianqihoubao.com/weather/city.aspx")

results= []

 # print driver.title
city_input = driver.find_element_by_id('txtareaName') 
city_input.clear()
city_input.send_keys('柯桥') 
time.sleep(0.2)


def get_first_last_weekday(year=2019,month=1,day=1):
    result = []
    weekday = datetime(year,month,day).weekday()+1
    start = weekday - 1
    day_count = (datetime(year,month+1,day) - datetime(year,month,day)).days
    result = [((start+i)//7 +1 , datetime(year,month,i+1).weekday()+1) for i in range(0, day_count)]
    return result

for month in tqdm([9,10,11]):
    month_week_list = get_first_last_weekday(year=2019,month=month,day=1)
    for row_index,col_index in tqdm(month_week_list):
        row_index +=1
        
        cal = driver.find_element_by_xpath('//*[@id="txtdate"]')
        cal.click()
        # time.sleep(10)
        # driver.find_element_by_class_name("sp_container").click()
        Select(driver.find_element_by_xpath("/html/body/div/div/table/thead/tr/th/select[1]")).select_by_value(str(month-1))
        # # time.sleep(0.2)
        # # Select(driver.find_element_by_class_name("sp_year")).select_by_visible_text("2019")
        # # time.sleep(1)
        # # deselect_all()
        Select(driver.find_element_by_xpath("/html/body/div/div/table/thead/tr/th/select[2]")).select_by_visible_text("2019")

        driver.find_element_by_xpath("/html/body/div/div/table/tbody/tr[{0}]/td[{1}]/a".format(row_index,col_index)).click()


        # js="document.getElementByClassName('sp_container').style.visibility='visible';";
        # driver.execute_script(js)
        # /html/body/div
        driver.find_element_by_xpath('//*[@id="btnSearch"]').click()
        
        date = driver.find_element_by_xpath('//*[@id="content"]/table[2]/tbody/tr/td[2]').text
        day_weather = driver.find_element_by_xpath('//*[@id="content"]/table[2]/tbody/tr/td[3]').text
        day_wind =  driver.find_element_by_xpath('//*[@id="content"]/table[2]/tbody/tr/td[4]').text
        day_tem =  driver.find_element_by_xpath('//*[@id="content"]/table[2]/tbody/tr/td[5]').text
        night_weather =  driver.find_element_by_xpath('//*[@id="content"]/table[2]/tbody/tr/td[6]').text
        night_wind =  driver.find_element_by_xpath('//*[@id="content"]/table[2]/tbody/tr/td[7]').text
        night_tem =  driver.find_element_by_xpath('//*[@id="content"]/table[2]/tbody/tr/td[8]').text
        
        result = [date,day_weather,day_wind,day_tem,night_weather,night_wind,night_tem]
        results.append(result)
results = pd.DataFrame(results,columns=['日期','白天天气','白天风向风力','白天温度','夜晚天气','夜晚风向风力','夜晚温度'])
results.to_csv('C:/Users/Flash/Desktop/keqiaoresult.csv',header=1,encoding='utf_8_sig',index=0) #保存列名



  0%|                                                                                            | 0/3 [00:00<?, ?it/s]


  0%|                                                                                           | 0/30 [00:00<?, ?it/s]


  3%|██▊                                                                                | 1/30 [01:04<31:17, 64.74s/it]


  7%|█████▌                                                                             | 2/30 [01:29<24:33, 52.62s/it]

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=78.0.3904.97)


In [ ]:
from selenium import webdriver
import time
import re
import pandas as pd
from tqdm import tqdm
# 要想调用键盘按键操作需要引入keys包
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select

keywords_list = ['G15','G92','主要路段有']
driver = webdriver.Chrome()
driver.get("http://www.mot.gov.cn/gongluchuxing/index.html")

driver.find_element_by_xpath('/html/body/div[2]/div[2]/div/div/div[1]/a[1]/span[1]').click() #点第一个
windows = driver.window_handles
time.sleep(0.5)
driver.switch_to.window(windows[-1])

text_elements = driver.find_elements_by_xpath("//p[contains(@align,'justify')]")  #找出align属性包含justify的元素。
for element in text_elements:
    texts = ""
    text = element.get_attribute('textContent')
    for keyword in keywords_list:
        if(text.find(keyword)!=-1):
            texts+=text
    text_list = texts.split('的主要路段有：')